In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV

from sklearn import linear_model, tree, ensemble, svm

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
file_path = 'data/customer_data.csv'
df = pd.read_csv(file_path)

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
def missing_value(dataframe):
    if dataframe.isnull().values.any():
        print(dataframe.isnull().sum())
    else:
        print("No Missing values")

In [ ]:
missing_value(df)

## Exploratory Data Analysis

In [ ]:
df.describe()

In [ ]:
df['TotalCharges'] = df['TotalCharges'].replace([' '], [0])
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='raise')

In [ ]:
heatdf = df.loc[:, ['tenure', 'MonthlyCharges', 'TotalCharges']]
sns.heatmap(heatdf.corr())
print(heatdf.corr())

In [ ]:
df_num = ['tenure', 'MonthlyCharges', 'TotalCharges']

# plot Numerical Data

a = 3 # number of rows
b = 2 # number of columns
c = 1 # initializa plot counter

for i in df_num:
    plt.subplot(a, b, c)
    plt.title('{} (dist), subplot: {}{}{}'.format(i, a, b, c))
    plt.xlabel(i)
    sns.distplot(heatdf[i])
    c = c + 1

    plt.subplot(a, b, c)
    plt.title('{} (box), subplot: {}{}{}'.format(i, a, b, c))
    plt.xlabel(i)
    plt.boxplot(x=heatdf[i])
    c = c + 1
plt.show()

In [ ]:
# Distribution of the continous variable with respect to the target variable
for i in df_num:
    fig, ax = plt.subplots()

    ax.hist(df[df['Churn']=='No'][i], bins=15, alpha=0.5, color="red", label="No")
    ax.hist(df[df['Churn']=='Yes'][i], bins=15, alpha=0.5, color="blue", label="Yes")

    ax.set_xlabel(i)
    ax.set_ylabel("Frequency")

    ax.legend()

In [ ]:
df['SeniorCitizen'] = df['SeniorCitizen'].replace([1, 0], ['Yes', 'No'])
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)

In [ ]:
df_cat = df.select_dtypes(include=['object'])
df_cat = df_cat.drop(columns=['customerID', 'Churn'])

In [ ]:
cat_col = list(df_cat.columns.values)

In [ ]:
cat_col

In [ ]:
count = 0
for i in cat_col:
    count+=1
print(count)

In [ ]:
df['Churn'].value_counts().plot.bar(title="Churn Distribution")
print(df['Churn'].value_counts())

In [ ]:
from turtle import title


for column in cat_col:

    df[column].value_counts().plot.bar(title=column)
    plt.tight_layout()
    plt.show()

In [ ]:
for i in cat_col:
    pd.crosstab(df[i], df.Churn).plot(kind='bar')
    plt.xlabel('{}'.format(i))
    plt.ylabel('Churn Status Frequency')
plt.show()

## Feature Engineering

In [ ]:
# Determining the number of Categories for the categories
object_cols = [col for col in df.columns in df[col].dtype == "object"]

# Get Number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: df[col].nunique(), object_cols))

d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(),key=lambda x: x[1])

In [ ]:
for var in cat_col:
    cat_list = 'var' + '_' + 'var'
    cat_list = pd.get_dummies(df[var], prefix=var)
    df_New = pd.concat([df,cat_list], axis=1)
    df = df_New

data_vars = df.columns.values.tolist()

to_keep = [i for i in data_vars if i not in cat_col]
df_final  = df[to_keep]
df_final.columns.values

## Modelling

In [ ]:
df_final['Churn'] = df_final['Churn'].replace(['Yes', 'No'], [1, 0])
df_model = df_final.drop(columns=['customerID'])
label = np.array(df_model['Churn'])

feature_df = df_model.drop('Churn', axis=1)

feature = np.array(feature_df)

In [ ]:
feature_train, feature_test, label_train, label_test = train_test_split(feature, label, test_size=0.2, random_state = 42)

In [ ]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

rf.fit(feature_train, label_train)


In [ ]:
y_pred = rf.predict(feature_test)

In [ ]:
acc = accuracy_score(label_test, y_pred)

In [ ]:
print('Baseline Model Accuracy: ', acc)

In [ ]:
from random import shuffle


kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state = 42)

count = 1

for train_index, test_index in kf.split(feature, label):
    print(f'Fold: {count}, Train set: {len(train_index)}, Validation set: {len(test_index)}')
    count += 1

In [ ]:
# Random forest after cross validation
score = cross_val_score(ensemble.RandomeForestClassifier(random_state = 42), feature, label, cv = kf, scoring="accuracy")
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.2f}".format(score.mean())}')

## Feature Importance Analysis

In [ ]:
gbc = ensemble.GradientBoostingClassifier(random_state=42)

In [ ]:
gbc.fit(feature, label)

In [ ]:
col_sorted_by_importance = gbc.feature_importances_.argsort()

feat_imp = pd.DataFrame({
    'cols': feature_df.columns[col_sorted_by_importance],
    'imps': gbc.feature_importances_[col_sorted_by_importance]
})
print(feat_imp.sort_values(by=['imps'], ascending=False).head())

## Selecting The Final Model

In [ ]:
model = GradientBoostingClassifier(random_state=1)
space = dict()
space['n_estimators'] = [10, 100, 500]
space['max_features'] = [2, 4, 6]

search = GridSearchCV(model, space, scoring='accuracy', cv = kf, refit=True)
result = search.fit(feature_train, label_train)

In [ ]:
best_model = result.best_estimator_

yhat = best_model.predict(feature_test)

acc = accuracy_score(label_test, yhat)
print(f'Best Model Accuracy: {"{:.3f}".format(acc)}')

In [ ]:
print("Confusion Matrix: ")
print(confusion_matrix(label_test, yhat))
print()
print("Classification")
print(classification_report(label_test, yhat))

In [ ]:
y_scores_gb = gbc.decision_function(feature_test)
fpr_gb, tpr_gb = roc_curve(label_test, y_scores_gb)
roc_auc_gb = auc(fpr_gb, tpr_gb)

print('Area under Roc curve = {:0.2f}'.format(roc_auc_gb))